# Homework: Not So Basic Artificial Neural Networks

Your task is to implement a simple framework for convolutional neural networks training. While convolutional neural networks is a subject of lecture 3, we expect that there are a lot of students who are familiar with the topic.

In order to successfully pass this homework, you will have to:

- Implement all the blocks in `homework_modules.ipynb` (esp `Conv2d` and `MaxPool2d` layers). Good implementation should pass all the tests in `homework_test_modules.ipynb`.
- Settle with a bit of math in `homework_differentiation.ipynb`
- Train a CNN that has at least one `Conv2d` layer, `MaxPool2d` layer and `BatchNormalization` layer and achieves at least 97% accuracy on MNIST test set.

Feel free to use `homework_main-basic.ipynb` for debugging or as source of code snippets. 

Note, that this homework requires sending **multiple** files, please do not forget to include all the files when sending to TA. The list of files:
- This notebook with cnn trained
- `homework_modules.ipynb`
- `homework_differentiation.ipynb`

In [78]:
%matplotlib inline
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [79]:
%pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


In [80]:
# (re-)load layers
%run homework_modules.ipynb

In [81]:
# batch generator
def get_batches(dataset, batch_size):
    X, Y = dataset
    n_samples = X.shape[0]
        
    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield X[batch_idx], Y[batch_idx]

In [82]:
import mnist
X_train, y_train, X_val, y_val, X_test, y_test = mnist.load_dataset()  # your dataset

In [12]:
# Your turn - train and evaluate conv neural network

In [83]:
n_classes = 10

In [84]:
nonlinearity = ReLU  # ELU

model = Sequential()

model.add(Conv2d(in_channels=1, out_channels=8, kernel_size=3))
model.add(Dropout(p=0.5))
model.add(nonlinearity())

model.add(MaxPool2d(kernel_size=2))  # size: [batch, 60, 7, 7]

model.add(Conv2d(in_channels=8, out_channels=16, kernel_size=3))
model.add(Dropout(p=0.5))
model.add(nonlinearity())

model.add(MaxPool2d(kernel_size=2))  # size: [batch, 60, 7, 7]

model.add(Flatten())  # size: [batch, 80 * 7 * 7]

model.add(Linear(n_in=16*7*7, n_out=450))
model.add(nonlinearity())
model.add(BatchNormalization(alpha=0.9))
model.add(ChannelwiseScaling(n_out=450))

model.add(Linear(n_in=450, n_out=300))
model.add(nonlinearity())
model.add(BatchNormalization(alpha=0.9))
model.add(ChannelwiseScaling(n_out=300))

model.add(Linear(n_in=300, n_out=200))
model.add(nonlinearity())
model.add(BatchNormalization(alpha=0.9))
model.add(ChannelwiseScaling(n_out=200))

model.add(Linear(n_in=200, n_out=100))
model.add(nonlinearity())
model.add(BatchNormalization(alpha=0.9))
model.add(ChannelwiseScaling(n_out=100))

model.add(Linear(n_in=100, n_out=n_classes))
model.add(LogSoftMax())

In [87]:
criterion = ClassNLLCriterion()

optimizer = adam_optimizer
state = {}  
config = {'learning_rate': 1e-2, 'beta1': 0.9, 'beta2':0.999, 'epsilon':1e-8}

n_epochs = 4

batch_size = 32
print_every = 100


In [88]:
model.train()

for epoch in range(1, n_epochs):
    print(f"_____\nepoch = {epoch}")
    config['learning_rate'] /= 10
    running_loss = 0
    for i in tqdm(range(len(X_train) // batch_size - 2)):
        x_batch = X_train[i * batch_size: (i + 1) * batch_size][:, np.newaxis]
        y_class_labels = y_train[i * batch_size: (i + 1) * batch_size]

        y_batch = np.zeros((batch_size, n_classes))
        y_batch[range(batch_size), y_class_labels] = 1
        
        log_preds = model.updateOutput(x_batch)
        loss = criterion.updateOutput(log_preds, y_batch)
        
        running_loss += loss
        grad = criterion.updateGradInput(log_preds, y_batch)
        model.backward(x_batch, grad)
        adam_optimizer(model.getParameters(), model.getGradParameters(), config, state)

        if not i % print_every and i:
            print(f"batch number {i}, loss = {running_loss / (print_every)}")
            running_loss = 0

  0%|          | 0/1560 [00:00<?, ?it/s]

_____
epoch = 1


  6%|▋         | 101/1560 [00:56<12:48,  1.90it/s]

batch number 100, loss = 1.9484223803424414


 13%|█▎        | 201/1560 [01:50<12:08,  1.86it/s]

...

 96%|█████████▌| 1491/1560 [12:46<00:41,  1.66it/s]

batch number 1500, loss = 0.12192008096630047


 96%|█████████▌| 1501/1560 [12:51<00:29,  1.98it/s]

batch number 1500, loss = 0.12192008096630047


100%|██████████| 1560/1560 [13:21<00:00,  1.95it/s]


Print here your accuracy on test set. It should be >97%. Don't forget to switch the network in 'evaluate' mode

In [89]:
model.evaluate()
y_pred = []
batch_size = 32
for i in tqdm(range(0, len(X_test) - batch_size, batch_size)):
    x_batch = X_test[i : i + batch_size][:, np.newaxis]   

    x_batch = x_orig
    y_pred += model.updateOutput(x_batch).tolist()

100%|██████████| 312/312 [01:07<00:00,  4.65it/s]
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 9984
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}


In [70]:
y_pred += model.updateOutput(X_test[len(y_pred):][:, np.newaxis]).tolist()

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 10000
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}


In [73]:
accuracy_score(y_test, np.argmax(y_pred, axis=1))

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 10000
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}


0.975